## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBSBB'
meas='PRCP'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
2658
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|-46.250465951354634|214.73042660811066|-199.57006811081482|   2001.0|SSSSBSBB| 32.9167|-105.4667|       PRCP|0.9958178383853654| 0.905669908003994|0.8278018402897551|0.9205614018580083|USC0029

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [14]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year     coeff_3
0  USC00295502  1924.0 -199.570068
1  US1NMDA0061  2010.0  -40.199825
2  USC00299697  1931.0  283.233313
3  USC00291910  2006.0   43.758868
4  USC00294009  2006.0 -132.827605

In [15]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  US1NMDA0011  US1NMDA0019  US1NMDA0024  US1NMDA0040  US1NMDA0041  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN          NaN   
1895.0           NaN          NaN          NaN          NaN          NaN   
1896.0           NaN          NaN          NaN          NaN          NaN   
1897.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   

station  US1NMDA0050  US1NMDA0061  US1NMDA0108  US1NMDA0112  US1NMDA0127  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN          NaN   
1895.0           NaN          NaN          NaN          NaN          NaN   
1896.0           NaN          NaN          NaN          NaN          NaN   
1897.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00299720  USC00299806  USS0008T01S  USW00003074  \
year        ...                                                            
1893.0      ...               NaN          NaN          NaN          NaN   
1895.0      ...               NaN          NaN          NaN          NaN   
1896.0      ...               NaN          NaN          NaN          NaN   
1897.0      ...               NaN          NaN          NaN          NaN   
1898.0      ...               NaN          NaN          NaN          NaN   
1899.0      ...               NaN          NaN          NaN          NaN   
1900.0      ...               NaN          NaN          NaN          NaN   
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   

station  USW00023002  USW00023039  USW00023046  USW00053026  USW00093045  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN          NaN   
1895.0           NaN          NaN          NaN          NaN          NaN   
1896.0           NaN          NaN          NaN          NaN          NaN   
1897.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   

station  USW00093063  
year                  
1893.0           NaN  
1895.0           NaN  
1896.0           NaN  
1897.0           NaN  
1898.0           NaN  
1899.0           NaN  
1900.0           NaN  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  

[10 rows x 140 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [16]:
def RMS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',RMS(year_station_table)
print 'MS removing mean-by-station= ',RMS(tbl_minus_station)
print 'MS removing mean-by-year   = ',RMS(tbl_minus_year)

total MS                   =  8712.13058304
MS removing mean-by-station=  7934.73707359
MS removing mean-by-year   =  3140.22083205


In [17]:
T=year_station_table
Init_MS=RMS(T)
print 'Total MS=',RMS(T)
for i in range(1):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    tempMS=RMS(T)
    print 'After removing mean-by-year    =',tempMS, ",fraction explained: ",(Init_MS-tempMS)/Init_MS*100.0
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    tempMS2=RMS(T)
    print 'After removing mean-by-stations=',RMS(T), ",fraction explained: ",(Init_MS-tempMS2)/Init_MS*100.0

Total MS= 8712.13058304
After removing mean-by-year    = 3140.22083205 ,fraction explained:  63.9557648715
After removing mean-by-stations= 2688.69560662 ,fraction explained:  69.1384836236
